# Predicting Evergreeness of Content with Decision Trees and Random Forests

In [1]:
## DATA DICTIONARY

In [2]:
import pandas as pd
import json
import itertools
import seaborn as sb
%matplotlib inline

data = pd.read_csv("assets/dataset/stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


In [4]:
list(data.columns)

['url',
 'urlid',
 'boilerplate',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'title',
 'body']

In [61]:
#Getting only non-string data
data_iter = data[['alchemy_category_score','avglinksize','commonlinkratio_1'\
                  ,'commonlinkratio_2','commonlinkratio_3','commonlinkratio_4','compression_ratio','embed_ratio',\
                  'framebased','frameTagRatio','hasDomainLink','html_ratio','image_ratio','is_news',
                  'lengthyLinkDomain','linkwordscore', 'news_front_page','non_markup_alphanum_characters',\
                  'numberOfLinks','numwords_in_url','parametrizedLinkRatio','spelling_errors_ratio','label']]
data_iter["is_news"][data_iter["is_news"]=="?"]=0
data_iter["alchemy_category_score"][data_iter["alchemy_category_score"]=="?"]=0
data_iter["news_front_page"][data_iter["news_front_page"]=="?"]=0
data_cols=list(data_iter.columns)
#I can do the full length of tags, but it will take forever, so I'm being lazy and only doing a few.
combo_len = range(1,5)

/Users/MP/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MP/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MP/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [52]:
#Creating a combination of the different features of different lengths
feature_list = []
for j in combo_len:
    for i in itertools.combinations(data_cols[:-1],j):
        feature_list.append(list(i))

In [53]:
#Importing SKLearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier


In [66]:
#Get the model from list we provide 
def make_model(features):
    model = DecisionTreeClassifier()
    X = data_iter[features].dropna()
    y = data_iter['label']
    fitted = model.fit(X,y)
    score = get_score(fitted,X,y,features)
    return score

#Get the Score
def get_score(model,X,y,features):
    scores=cross_val_score(model, X, y, scoring='roc_auc', cv=5)
    #print "For the variables: ",features
    #print "Average AUC: ",scores.mean()
    #print "---------------------"
    return scores.mean()

In [67]:
topscore=0
for k in feature_list:
    new_score = make_model(k)
    if new_score > topscore:
        topscore = new_score
        best_feat = k
print "The Best Score Is:  ",topscore
print "With variables: ",k

KeyboardInterrupt: 

###  Adjusting Decision Trees to Avoid Overfitting

 ### Demo: Control for overfitting in the decision model by adjusting the maximum number of questions (max_depth) or the minimum number of records in each final node (min_samples_leaf)

In [12]:
model = DecisionTreeClassifier(
                max_depth = 2,
                min_samples_leaf = 5)

model.fit(X, y)
build_tree_image(model)

 ### Demo: Build a random forest model to predict the evergreeness of a website. 

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
    
model.fit(X, y)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

### Demo: Extracting importance of features

In [14]:
features = X.columns
feature_importances = model.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)

features_df.head()

,Features,Importance Score
1,html_ratio,0.525093
0,image_ratio,0.446560
2,recipe,0.028348


 ### Exercise: Evaluate the Random Forest model using cross-validation; increase the number of estimators and view how that improves predictive performance.

In [15]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

for n_trees in range(1, 100, 10):
    model = RandomForestClassifier(n_estimators = n_trees)
    scores = cross_val_score(model, X, y, scoring='roc_auc')
    print('n trees: {}, CV AUC {}, Average AUC {}'.format(n_trees, scores, scores.mean()))

CV AUC [ 0.57570238  0.58915353  0.57742601], Average AUC 0.580760641743
n trees: 1, CV AUC [ 0.54255906  0.53819544  0.5617349 ], Average AUC 0.547496466645
n trees: 11, CV AUC [ 0.56313364  0.57108026  0.57715482], Average AUC 0.570456237596
n trees: 21, CV AUC [ 0.56348595  0.58999258  0.57831501], Average AUC 0.57726451696
n trees: 31, CV AUC [ 0.56784844  0.58441561  0.57399503], Average AUC 0.575419696308
n trees: 41, CV AUC [ 0.56541757  0.58635678  0.58181446], Average AUC 0.577862937957
n trees: 51, CV AUC [ 0.56831534  0.5876955   0.58261879], Average AUC 0.579543211076
n trees: 61, CV AUC [ 0.56778802  0.58738123  0.58584902], Average AUC 0.580339424781
n trees: 71, CV AUC [ 0.56860293  0.58629003  0.58255265], Average AUC 0.579148535326
n trees: 81, CV AUC [ 0.56522012  0.58790072  0.58500996], Average AUC 0.579376936253
n trees: 91, CV AUC [ 0.56998115  0.59060494  0.58089139], Average AUC 0.580492495925


##  Independent Practice: Evaluate Random Forest Using Cross-Validation

1. Continue adding input variables to the model that you think may be relevant
2. For each feature:
  - Evaluate the model for improved predictive performance using cross-validation
  - Evaluate the _importance_ of the feature
  - 
3. **Bonus**: Just like the 'recipe' feature, add in similar text features and evaluate their performance.


In [17]:
## 1. Building a model with more relevant features

model = RandomForestClassifier(n_estimators=50)

# Continue to add features to X
#     Build dummy features, include quantitative features, or add text features
X = data[['image_ratio', 'html_ratio', 'recipe', 'label']].dropna()

y = X['label']

X.drop('label', axis=1, inplace=True)


## 2a. Evaluate predictive performance for the given feature set
scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

# 3 (BONUS): Adding in text features

# Check for keywords in the title
data['PhotoInTitle'] = data['title'].fillna('').str.lower().str.contains('photo').astype(int)
X = data[['image_ratio', 'html_ratio', 'recipe', 'PhotoInTitle', 'label']].dropna()
X.drop('label', axis=1, inplace=True)


scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))


## 2b. Evaluating feature importances

# Fit a model on the whole dataset
model.fit(X, y)

# Get columns and their scores
features = X.columns
feature_importances = model.feature_importances_
features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)

features_df.head()

CV AUC [ 0.56969587  0.59154016  0.58402571], Average AUC 0.581753915204
CV AUC [ 0.56678292  0.59568423  0.59365489], Average AUC 0.585374012409


,Features,Importance Score
1,html_ratio,0.526194
0,image_ratio,0.438206
2,recipe,0.030021
3,PhotoInTitle,0.005579
